In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)
import numpy as np
import math
import pandas
from matplotlib import pyplot as plt
from PIL import Image
import os

In [1]:
all_photos = np.load('KnightPhotos.npy')

all_labels = np.load('KnightLabels.npy') # Эта штука соответствует одному из значений в all_degrees, котрое соответствует некой маске
all_vectors = np.load('KnightVectors.npy')

all_degrees = np.load('KnightRotationDegrees.npy')
all_albedos = np.load('KnightAlbedos.npy')
all_normals = np.load('KnightNormals.npy')
all_masks = np.load('KnightMasks.npy')

NameError: name 'np' is not defined

In [ ]:
def ShowImageSeriesUpTo16(imageList): # input is normal LIST
    if len(imageList) > 64: 
        return
    while len(imageList) % 4 != 0:
        imageList.append(np.ones(imageList[0].shape))
    readyRows = list()
    for y in range(len(imageList) // 4):
        readyRows.append(np.concatenate((imageList[y * 4 +0],imageList[y * 4 +1],imageList[y * 4 +2],imageList[y * 4 +3]), axis = 1))

    final = np.concatenate(readyRows, axis = 0)
    return final
myArray = [all_photos[0], all_albedos[0], all_normals[0], all_masks[0]]
ExpandedImage = ShowImageSeriesUpTo16(myArray).astype('float32')
print(ExpandedImage.shape)
plt.imshow(ExpandedImage)

In [ ]:
def ManualCheck(original_index):
    #print(all_photos.shape)
    inputt = all_photos[original_index].reshape(1, 64, 64, 3)
    prediction = model.predict(inputt)[0].round(3)
    origV = (all_vectors[original_index]).round(3)
    predV = (prediction).round(3)
    print("Prediction  : ", predV)
    print("Ground truth: ", origV)
    print("Diff ", origV - predV)
    print("Total Diff ", np.sum(origV - predV))
    
    print("orig L: ", np.linalg.norm(origV))
    print("pred L: ", np.linalg.norm(predV))
    plt.imshow(all_photos[original_index])

In [ ]:
train_size = int(0.95 * len(all_vectors))

dataset = tf.data.Dataset.from_tensor_slices((all_photos, all_vectors))
dataset = dataset.shuffle(buffer_size=128, seed=42)

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

train_dataset = train_dataset.batch(32)
test_dataset = test_dataset.batch(32)

print(len(list(train_dataset)), len(list(test_dataset)))

In [ ]:
tf.keras.utils.set_random_seed(42)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(4, (5, 5), activation='linear', input_shape=(64, 64, 3)),
  #tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(4, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(4, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(4, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(4, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(4, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(5, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(5, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(6, (5, 5), activation='linear'),
  tf.keras.layers.Conv2D(6, (5, 5), activation='linear'),

  tf.keras.layers.Flatten(),

  tf.keras.layers.Dense(32, activation='linear'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(24, activation='linear'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(16, activation='linear'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(3)
])
model.compile(optimizer='adam',
              loss='mean_absolute_error', #mean_squared_error
              metrics=['mean_absolute_error'])             
model.summary()

In [ ]:
tf.keras.utils.set_random_seed(42)
model.fit(train_dataset, epochs=5)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
ManualCheck(2500)